# Address Matching Deep Dive

This notebook experiments with different ways of comparing addresses in order to demonstrate the power of parsed address comparison.

In [1]:
import json
import logging
import os
import re
import sys
import time
import warnings
from numbers import Number
from typing import Callable, Dict, List, Literal, Sequence, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pycountry
import pytest
import random
import seaborn as sns
import torch
import torch.nn as nn
import wandb
from datasets import Dataset
from fuzzywuzzy import fuzz

from postal.parser import parse_address
from scipy.spatial import distance
from sklearn.metrics import (
    accuracy_score,
    precision_recall_curve,
    precision_recall_fscore_support,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve,
    auc
)
from sklearn.model_selection import train_test_split
from sentence_transformers import InputExample, SentenceTransformer, SentencesDataset, SentenceTransformerTrainer, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, SimilarityFunction, BinaryClassificationEvaluator
from sentence_transformers.model_card import SentenceTransformerModelCardData
from sentence_transformers.training_args import BatchSamplers, SentenceTransformerTrainingArguments
from tenacity import retry
from torch.utils.data import DataLoader
from tqdm.autonotebook import tqdm
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, TrainingArguments, Trainer
from transformers.integrations import WandbCallback

from utils import (
    augment_gold_labels,
    compute_sbert_metrics,
    compute_classifier_metrics,
    gold_label_report,
    preprocess_logits_for_metrics,
    to_dict,
    prettify_df,
)

In [2]:
logging.basicConfig(stream=sys.stderr, level=logging.ERROR)

logger = logging.getLogger(__name__)

#### Ignore Warnings

In [3]:
warnings.simplefilter('ignore', FutureWarning)

#### Configure Weights & Biases

`wandb` needs some environment variables to work.

In [4]:
os.environ["WANDB_LOG_MODEL"] = "end"
os.environ["WANDB_WATCH"] = "all"
os.environ["WANDB_PROJECT"] = "libpostal-reborn"
os.environ["WANDB_DISABLED"] = "false"
os.environ["WANDB_IGNORE_GLOBS"] = ".env"

#### Configure Huggingface APIs

In [5]:
os.environ["HF_ENDPOINT"] = "https://huggingface.co/"

#### Configure Huggingface APIs

Squash any warnings...

In [6]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

#### Configure Pandas to Show More Rows

In [7]:
pd.set_option("display.max_rows", 40)
pd.set_option("display.max_columns", None)

### Use CUDA or MPS if Avaialable

CPU training and even inference with sentence transformers and deep learning models is quite slow. Since all machine learning in this library is based on [PyTorch](https://pytorch.org/get-started/locally/), we can assign all ML operations to a GPU in this one block of code. Otherwise we default to CPU without acceleration. The notebook is still workable in this mode, you just may need to grab a cup of tea or coffee while you wait for it to train the Sentence-BERT model below.

In [8]:
# Check for CUDA or MPS availability and set the device
if torch.backends.mps.is_available():
    device = torch.device("mps")
    logger.debug("Using Apple GPU acceleration")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    logger.debug("Using NVIDIA CUDA GPU acceleration")
else:
    device = "cpu"
    logger.debug("Using CPU for ML")

device

device(type='mps')

### Use Weights & Biases for Logging Metrics

Weights & Biases has a free account for individuals with public projects. Using it will produce charts during our training runs that anyone can view. You can create your own project for this notebook and login with that key to log your own training runs.

You may need to run the following command from your shell before the next cell, otherwise you will have to paste your project key into the 

```bash
wandb login
```

In [9]:
# Login to wandb. Comment out if you already haven't via `wandb login` from a CLI
wandb.login()

wandb: Currently logged in as: rjurney. Use `wandb login --relogin` to force relogin


True

## Implementing a Structured Address Matcher

Let's start our exercise by using the structured address data provided by [Libpostal](https://github.com/openvenues/libpostal) to parse them for matching. We write a function for each address part to deal with missing fields without duplicating a lot of logic.

We start with something quite literal and basic. We'll improve it as we go.

In [10]:
def parse_match_address(address1: str, address2: str) -> Literal[0, 1]:
    """parse_match_address implements address matching using the precise, parsed structure of addresses."""
    address1 = to_dict(parse_address(address1))
    address2 = to_dict(parse_address(address2))

    def match_road(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_road - literal road matching, negative if either lacks a road"""
        if ("road" in address1) and ("road" in address2):
            if address1["road"] == address2["road"]:
                logger.debug("road match")
                return 1
            else:
                logger.debug("road mismatch")
                return 0
        logger.debug("road mismatch")
        return 0

    def match_house_number(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_house_number - literal house number matching, negative if either lacks a house_number"""
        if ("house_number" in address1) and ("house_number" in address2):
            if address1["house_number"] == address2["house_number"]:
                logger.debug("house_number match")
                return 1
            else:
                logger.debug("house_number mismatch")
                return 0
        logger.debug("house_number mistmatch")
        return 0

    def match_unit(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_unit - note a missing unit in both is a match"""
        if "unit" in address1:
            if "unit" in address2:
                logger.debug("unit match")
                return 1 if (address1["unit"] == address2["unit"]) else 0
            else:
                logger.debug("unit mismatch")
                return 0
        if "unit" in address2:
            if "unit" in address1:
                logger.debug("unit match")
                return 1 if (address1["unit"] == address2["unit"]) else 0
            else:
                logger.debug("unit mismatch")
                return 0
        # Neither address has a unit, which is a default match
        return 1

    def match_postcode(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_postcode - literal matching, negative if either lacks a postal code"""
        if ("postcode" in address1) and ("postcode" in address2):
            if address1["postcode"] == address2["postcode"]:
                logger.debug("postcode match")
                return 1
            else:
                logger.debug("postcode mismatch")
                return 0
        logger.debug("postcode mismatch")
        return 0

    def match_country(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_country - literal country matching - pass if both don't have one"""
        if ("country" in address1) and ("country" in address2):
            if address1["country"] == address2["country"]:
                logger.debug("country match")
                return 1
            else:
                logger.debug("country mismatch")
                return 0
        # One or none countries should match
        logger.debug("country match")
        return 1

    # Combine the above to get a complete address matcher
    if (
        match_road(address1, address2)
        and match_house_number(address1, address2)
        and match_unit(address1, address2)
        and match_postcode(address1, address2)
        and match_country(address1, address2)
    ):
        logger.debug("overall match")
        return 1
    else:
        logger.debug("overall mismatch")
        return 0

In [11]:
# Yup down to house_number ...
parse_match_address(
    "3413 Sean Way, Lawrenceville, GA 30044",
    "3413 Sean Way Lawrenceville, GA, 30044",
)

1

In [12]:
fuzz.ratio(
    "3413 Sean Way, Lawrenceville, GA 30044",
    "3413 Sean Way Lawrenceville, GA, 30044",
)

97

In [13]:
# Yup down to unit ...
parse_match_address(
    "120 Ralph McGill Blvd, Apt 101, Atlanta, GA 30308, USA",
    "120 Ralph McGill Blvd, Apt 101, Atlanta GA 30308",
)

1

In [14]:
fuzz.ratio(
    "120 Ralph McGill Blvd, Apt 101, Atlanta, GA 30308, USA",
    "120 Ralph McGill Blvd, Apt 101, Atlanta GA 30308",
)

94

In [15]:
# Nope if only one uses an abbreviation ...
parse_match_address(
    "120 Ralph McGill Blvd, Apt 333, Atlanta, GA 30308",
    "120 Ralph McGill Boulevard, Apt 333, Atlanta, GA 30308",
)

0

In [16]:
fuzz.ratio(
    "120 Ralph McGill Blvd, Apt 333, Atlanta, GA 30308",
    "120 Ralph McGill Boulevard, Apt 333, Atlanta, GA 30308",
)

95

In [17]:
# Nope if one character in the streetname is off ...
parse_match_address(
    "120 Ralp McGill Blvd, Apt 333, Atlanta, GA 30308",
    "120 Ralph McGill Blvd, Apt 333, Atlanta, GA 30308",
)

0

In [18]:
fuzz.ratio(
    "120 Ralp McGill Blvd, Apt 333, Atlanta, GA 30308",
    "120 Ralph McGill Blvd, Apt 333, Atlanta, GA 30308",
)

99

### Literal is Too Precise!

While it is useful to parse addresses and implement literal matching logic as we did above, as the third example indicates, an abbreviation or a single typo results in a mistmatch. We're going to write a more complex, approximate logical matcher below using string distance and text embeddings.

Depending on your application you might relax this criteria to include corner cases such as missing postcodes. Before we get into that, let's create some training and evaluation data using hand-labeled data with an LLM data augmentation strategy to generate a lot of labeled records.

## Data Augmentation with the OpenAI GPT4o API

We need training data for our supervised learning approaches to addres matching. Open the sister notebook [Address Data Augmentation.ipynb](Address%20Matching%20Deep%20Dive.ipynb) before procceeding to further cells in order to create some training data via minimal manual labeling and programmatic data labeling for data augmentation. This will teach you programmatic data labeling, a critical skill that LLMs make MUCH EASIER because they understand things like the semantics of global addresses.

In [19]:
gold_df = pd.read_csv("data/gold.csv")

In [20]:
# If you want to start from here and not run the data augmentation pipeline again...
augment_results_df = pd.read_parquet("data/training.6.parquet")

augment_results_df.head(20)

,Address1,Address2,Description,Label
0,"123 E Main St, Springfield, IL 62701","123 East Main Street, Springfield, Illinois 62701",Different directional prefix formats for same ...,1.0
1,"456 W Elm St, Boston, MA 02118","456 West Elm Street, Boston, Massachusetts 02118",Different directional prefix formats for same ...,1.0
2,"789 S Oak St, Denver, CO 80203","789 South Oak Street, Denver, Colorado 80203",Different directional prefix formats for same ...,1.0
3,"321 N Pine St, Seattle, WA 98101","321 North Pine Street, Seattle, Washington 98101",Different directional prefix formats for same ...,1.0
4,"654 E Maple St, Austin, TX 73301","654 East Maple Street, Austin, Texas 73301",Different directional prefix formats for same ...,1.0
5,"987 W Cedar St, San Francisco, CA 94102","987 West Cedar Street, San Francisco, Californ...",Different directional prefix formats for same ...,1.0
6,"246 S Birch St, New York, NY 10001","246 South Birch Street, New York, New York 10001",Different directional prefix formats for same ...,1.0
7,"135 N Cedar Ave, Chicago, IL 60601","135 North Cedar Avenue, Chicago, Illinois 60601",Different directional prefix formats for same ...,1.0
8,"864 E Pine Ave, Los Angeles, CA 90001","864 East Pine Avenue, Los Angeles, California ...",Different directional prefix formats for same ...,1.0
9,"753 W Spruce St, Houston, TX 77001","753 West Spruce Street, Houston, Texas 77001",Different directional prefix formats for same ...,1.0


### Data Augmentation Complete!

Starting by hand labeling under 100 records and iterating a few times on data augmentation instructions for GPT4o, we have multiplied them by many times to get almost 10,000 synthetic records! This is enough to fine-tune a `SentenceTransformer` or semantic text similarity classifier model. GPT4o is a powerful tool for data augmentation! This can work for a variety of problems.

LLM based data augmentation is a powerful tool for your data labeling toolbox.

# Comparing Different Approaches to Address Matching

Now we're going to compare the following methods of address matching:

1) String Distance - we'll use PyPI library [fuzzywuzzy](https://pypi.org/project/fuzzywuzzy/0.6.1/) to compute the Levenshtein ratio and partial ratio of how many edits are required to match the address strings. There are times this works well, and there are times it couldn't be more wrong.
2) Text Embeddings - we'll use transfer learning to load an existing [SentenceTransformer](https://sbert.net) model to sentence encode pairs of addresses to create fixed-length embeddings for each address and then compute a similarity score via [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). This won't work without fine-tuning, so we fine-tune the model to the task.
3) Deep Matching Model - We'll train a deep semantic textual similarity classification model that will use Siamese BERT networks as in [Sentence-BERT](https://arxiv.org/abs/1908.10084) to classify address pairs as matching or not matching.

## Imprecise Country Matching with `pycountry`

The structured address has fields that each have their own semantics. Tools specific to a field can help match address components.

If you have any valid ISO nation abbreviation or long form name, [pycountry](https://pypi.org/project/pycountry/) is a PyPi module that can retrieve the actual country for it. This enables efficient comparison. Let's use that to convert the mismatched Singapore references in these records to the same entity.

In [ ]:
def match_country_names(country1: str, country2: str) -> Literal[0, 1]:
    """match_country_strings - compare and match varying country formats using pycountry"""

    # Remove any punctuation from the country
    def remove_punctuation(country: str) -> str:
        # Use re.sub to replace all punctuation characters with an empty string
        return re.sub(r"[^\w\s]", "", country)

    def multi_lookup(**kwargs):
        """Try each key until we retrieve a result"""
        for arg, value in kwargs.items():
            result = pycountry.countries.get(**{arg: value})
            if result:
                return result

    def get_args(country: str):
        """Compose pycountries.countries.get arguments dict based on length of country string"""
        args = {}
        if country and len(country) == 2:
            args["alpha_2"] = country
        elif country and len(country) == 3:
            args["alpha_3"] = country
        elif country:
            args["name"] = country
            args["common_name"] = country
            args["official_name"] = country
        return args

    try:
        pycountry1 = multi_lookup(**get_args(remove_punctuation(country1)))
        pycountry2 = multi_lookup(**get_args(remove_punctuation(country2)))

        return 1 if pycountry1.name == pycountry2.name else 0
    except AttributeError:
        return 0

In [ ]:
match_country_names("sg", "singapore")

In [ ]:
match_country_names("usa", "united states")

In [ ]:
# Didn't match until I added and called remove_punctuation(country: str) -> str
match_country_names("U.S.A.", "United States of America")

In [ ]:
match_country_names("USA", "MEX")

In [ ]:
match_country_names("United States", "United Mexican States")

### Country Parsing in Structured Matching

Let's use our new method `match_pycountry(country1: str, country2: str) -> Literal[0, 1]` matcher to improve our original structured matcher. This will allow it to contain varying country formats and still match. This makes the matcher more robust. 

In order to make this work we have to refactor our code to create matching functions for each field. Note that we are leaving out matching states, as they aren't required if the road name, number, unit and postal code match.

In [ ]:
def parse_match_address_country(address1: str, address2: str) -> Literal[0, 1]:
    """parse_match_address_country implements address matching like parse_match_address() but with pycountry country matching"""
    address1 = to_dict(parse_address(address1))
    address2 = to_dict(parse_address(address2))

    def match_road(address1: str, address2: str) -> Literal[0, 1]:
        """match_road - literal road matching, negative if either lacks a road"""
        if ("road" in address1) and ("road" in address2):
            if address1["road"] == address2["road"]:
                logger.debug("road match")
                return 1
            else:
                logger.debug("road mismatch")
                return 0
        logger.debug("road mismatch")
        return 0

    def match_house_number(address1: str, address2: str) -> Literal[0, 1]:
        """match_house_number - literal house number matching, negative if either lacks a house_number"""
        if ("house_number" in address1) and ("house_number" in address2):
            if address1["house_number"] == address2["house_number"]:
                logger.debug("house_number match")
                return 1
            else:
                logger.debug("house_number mismatch")
                return 0
        logger.debug("house_number mistmatch")
        return 0

    def match_unit(address1: str, address2: str) -> Literal[0, 1]:
        """match_unit - note a missing unit in both is a match"""
        if "unit" in address1:
            if "unit" in address2:
                logger.debug("unit match")
                return 1 if (address1["unit"] == address2["unit"]) else 0
            else:
                logger.debug("unit mismatch")
                return 0
        if "unit" in address2:
            if "unit" in address1:
                logger.debug("unit match")
                return 1 if (address1["unit"] == address2["unit"]) else 0
            else:
                logger.debug("unit mismatch")
                return 0
        # Neither address has a unit, which is a default match
        return 1

    def match_postcode(address1: str, address2: str) -> Literal[0, 1]:
        """match_postcode - literal matching, negative if either lacks a postal code"""
        if ("postcode" in address1) and ("postcode" in address2):
            if address1["postcode"] == address2["postcode"]:
                logger.debug("postcode match")
                return 1
            else:
                logger.debug("postcode mismatch")
                return 0
        logger.debug("postcode mismatch")
        return 0

    def match_country(address1: str, address2: str) -> Literal[0, 1]:
        """match_country - semantic country matching with pycountry via match_country_names(country1, country2)"""
        if ("country" in address1) and ("country" in address2):
            if match_country_names(address1["country"], address2["country"]):
                logger.debug("country match")
                return 1
            else:
                logger.debug("country mismatch")
                return 0
        # One or none countries should match
        logger.debug("country match")
        return 1

    # Combine the above to get a complete address matcher
    if (
        match_road(address1, address2)
        and match_house_number(address1, address2)
        and match_unit(address1, address2)
        and match_postcode(address1, address2)
        # Our only non-exact match - default to 1, match
        and match_country(address1, address2)
    ):
        logger.debug("overall match")
        return 1
    else:
        logger.debug("overall mismatch")
        return 0

In [ ]:
parse_match_address_country(
    "100 Roxas Blvd, Ermita, Manila, 1000 Metro Manila, PH",
    "100 Roxas Blvd, Ermita, Manila, 1000 Metro Manila, Republic of the Philippines"
)

In [ ]:
# Defaults to match if no countries are provided
parse_match_address_country(
    "100 King St W, Toronto, ON M5X 1A9",
    "100 King St W, Toronto, ON M5X 1A9",
)

In [ ]:
# Defaults to match if only one address has country
parse_match_address_country(
    "100 King St W, Toronto, ON M5X 1A9",
    "100 King St W, Toronto, ON M5X 1A9, Canada",
)

In [ ]:
# Verify mismatch
parse_match_address_country(
    "Bosque de Chapultepec I Secc, Miguel Hidalgo, 11850 Ciudad de México, CDMX, Mexico",
    "Bosque de Chapultepec I Secc, Miguel Hidalgo, 11850 Ciudad de México, CDMX, USA"
)

### Gold Label Validation

We need to evaluate this new method against our gold labeled data.

In [ ]:
def strict_parse_match(row: pd.Series) -> pd.Series:
    """strict_parse_match Strict address matching"""
    return parse_match_address(row["Address1"], row["Address2"])

In [ ]:
def parse_match_country(row: pd.Series) -> pd.Series:
    """parse_match Strict address matching"""
    return parse_match_address_country(row["Address1"], row["Address2"])

In [ ]:
raw_df, grouped_df = gold_label_report(gold_df, [strict_parse_match, parse_match_country])

In [ ]:
grouped_df

In [ ]:
true_df = raw_df[raw_df["parse_match_country_correct"]]
print(f"Total accurate matches for strict_parse_match: {len(true_df):,}")

true_df.sort_values(by="Description").reset_index(drop=True)

In [ ]:
false_df = raw_df[raw_df["parse_match_country_correct"] == False]
print(f"Total mismatches for strict_parse_match: {len(false_df):,}")

false_df.sort_values(by="Description").reset_index(drop=True)

# Machine Learning Approaches to Address Matching

In this section we pursue two machine learning approaches to address matching, in order of sophistication. First we fine-tune a pre-trained embedding model to our task, try it on our data and search for a threshold similarity that results in good performance for our address matching problem. Second we build a Siamese BERT network model based on [Sentence-BERT](https://arxiv.org/abs/1908.10084) to classify pairs of addresses as match or mismatch. We will train it using the same dataset we use to fine-tune a sentence transformer, and if we have enough training data this will likely be a more powerful approach.

## Text Embeddings, Sentence Encoding, `SentenceTransformers`, Vector Distance and Cosine Similarity

Text embeddings are trained on large volumes of text that include addresses. As a result they have some understanding of address strings and can do a form of semantic comparison that is less explicit than logical comparisons with address parsing. They're an important benchmark to explore. Huggingface has an excellent [introduction to sentence similarity](https://huggingface.co/tasks/sentence-similarity).

In our first machine learning approach, we are going to use transfer learning to load a pre-trained [sentence transformer](https://sbert.net) models from huggingface. We will use the training data we've prepared to fine-tune this model to our task, before rigorously evaluating it along with our other approaches.

Sentence transformers sentence encode strings of different distances into fixed-length vectors, a technique called sentence encoding. Once two address strings are embedded into a pair of equal length vectors, they can be compared with cosine similarity to get a distance, the inverse of which is a similarity score.

### Convert our `pd.DataFrame` to a `List[sentence_transformers.InputExample]`

First we need to convert our Pandas `DataFrame` to a list of sentence transformer input examples. `InputExamples` require two fields `texts=List[str, str]` and `label`.

In [21]:
train_df, tmp_df = train_test_split(augment_results_df, test_size=0.2, shuffle=True)
eval_df, test_df = train_test_split(tmp_df, test_size=0.5, random_state=42, shuffle=True)

train_dataset = Dataset.from_dict({
    "sentence1": train_df["Address1"].tolist(),
    "sentence2": train_df["Address2"].tolist(),
    "label": train_df["Label"].tolist(),
})

eval_dataset = Dataset.from_dict({
    "sentence1": eval_df["Address1"].tolist(),
    "sentence2": eval_df["Address2"].tolist(),
    "label": eval_df["Label"].tolist(),
})

test_dataset = Dataset.from_dict({
    "sentence1": test_df["Address1"].tolist(),
    "sentence2": test_df["Address2"].tolist(),
    "label": test_df["Label"].tolist(),
})

print(f"Training data:   {len(train_df):,}")
print(f"Validation data: {len(eval_df):,}")
print(f"Test data        {len(eval_df):,}")

Training data:   8,024
Validation data: 1,003
Test data        1,003


### Configure Fine-Tuning, Initialize a `SentenceTransformer`

To use the training data we prepared to fine-tune a `SentenceTransformer`, we need to select and load a pre-trained model from Huggingface Hub. Here are some models you can try:

* [sentence-transformers/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) - multilingual paraphrase models are designed to compare sentences in terms of their semantics.
* [BAAI/bge-m3](https://huggingface.co/BAAI/bge-m3) - a robust, multilingual model optimized for a variety of tasks
* [sentence-transformers/paraphrase-multilingual-mpnet-base-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2) - MPNet is another paraphrase model architecture we can fine-tune for address comparison
* [sentence-transformers/all-mpnet-base-v2](https://huggingface.co/sentence-transformers/all-mpnet-base-v2) - a top performing MPNet model

In [22]:
SBERT_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"
VARIANT = "original"
MODEL_SAVE_NAME = (SBERT_MODEL + "-" + VARIANT).replace("/", "-")

# Make sure these match the values in the data augmentation notebook for accurate loggging and reporting
CLONES_PER_RUN = 100
RUNS_PER_EXAMPLE = 2

EPOCHS = 6
BATCH_SIZE = 32
PATIENCE = 3
LEARNING_RATE = .00005
DATASET_MULTIPLE = CLONES_PER_RUN * RUNS_PER_EXAMPLE
SBERT_OUTPUT_FOLDER = f"data/fine-tuned-sbert-{MODEL_SAVE_NAME}"
SAVE_EVAL_STEPS = 100

### Initialize Weights & Biases

Weights and biases `wandb` package makes it simple to monitor the performance of your training runs.

In [23]:
# Initialize Weights & Biases
wandb.init(
    entity="rjurney",
    # set the wandb project where this run will be logged
    project="libpostal-reborn",
    # track hyperparameters and run metadata
    config={
        "variant": VARIANT,
        "dataset_multiple": DATASET_MULTIPLE,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "patience": PATIENCE,
        "learning_rate": LEARNING_RATE,
        "sbert_model": SBERT_MODEL,
        "sbert_output_folder": SBERT_OUTPUT_FOLDER,
        "save_eval_steps": SAVE_EVAL_STEPS,
    },
)

### Setup our `SentenceTransformer` Model

Choose the model to fine-tune above in `SBERT_MODEL` and instantiate it below.

In [24]:
sbert_model = SentenceTransformer(
    SBERT_MODEL,
    device=device,
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name=f"{SBERT_MODEL}-address-matcher-{VARIANT}",
    ),
)

### Evaluate our Model Before Fine-Tuning

Let's see what it can do without fine-tuning, then we'll compare our subjective results afterwards. This won't work very well, fine-tuning is required!

In [25]:
def sbert_compare(address1: str, address2: str) -> float:
    """sbert_compare - sentence encode each address into a fixed-length text embedding.
    Fixed-length means they can be compared with cosine similarity."""
    embedding1 = sbert_model.encode(address1)
    embedding2 = sbert_model.encode(address2)

    # Compute cosine similarity
    return 1 - distance.cosine(embedding1, embedding2)


def sbert_match(row: pd.Series) -> pd.Series:
    """sbert_match - SentenceTransformer address matching, float iytoyt"""
    return sbert_compare(row["Address1"], row["Address2"])


def sbert_compare_binary(address1: str, address2: str, threshold: float = 0.5) -> Literal[0, 1]:
    """sbert_match - compare and return a binary match"""
    similarity = sbert_compare(address1, address2)
    return 1 if similarity >= threshold else 0


def sbert_match_binary(row: pd.Series, threshold: float = 0.5) -> pd.Series:
    """sbert_match_binary - SentenceTransformer address matching, binary output"""
    return sbert_compare_binary(row["Address1"], row["Address2"], threshold=threshold)

In [26]:
# Still too similar - very hard to train them away from this behavior!
sbert_compare(
    "101 Oak Lane, Atlanta, GA 30308",
    "102 Oak Lane, Atlanta, GA 30308",
)

0.9887310599046897

In [27]:
# A little bit further away ...
sbert_compare(
    "101 Oak Lane, Atlanta, GA 30308",
    "101 Oak Ln., Atlanta, GA 30308",
)

0.9080112054372381

In [28]:
# Properly distant ...
sbert_compare(
    "3413 Sean Way, Lawrenceville, GA 30044",
    "1202 Oak Rd., Lawrenceville, GA 30304",
)

0.7771668960380212

In [29]:
# Properly similar ...
sbert_compare(
    "3413 Sean Way, Lawrenceville, GA 30044",
    "3413 Sean Way, Lawrenceville, GA 30044, USA",
)

0.9754176576820044

### Evaluate the Test Set with the Untrained Model

Let's see how well the [paraphrase-multilingual-MiniLM-L12-v2](https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2) model does on its own. This is our baseline score.

In [30]:
# Initialize the evaluator
binary_acc_evaluator = BinaryClassificationEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    labels=eval_dataset["label"],
    name=SBERT_MODEL,
)
binary_acc_evaluator(sbert_model)

{'paraphrase-multilingual-MiniLM-L12-v2_cosine_accuracy': 0.5653040877367896,
 'paraphrase-multilingual-MiniLM-L12-v2_cosine_accuracy_threshold': 0.7163583636283875,
 'paraphrase-multilingual-MiniLM-L12-v2_cosine_f1': 0.6951048951048951,
 'paraphrase-multilingual-MiniLM-L12-v2_cosine_f1_threshold': 0.7163583636283875,
 'paraphrase-multilingual-MiniLM-L12-v2_cosine_precision': 0.5361380798274002,
 'paraphrase-multilingual-MiniLM-L12-v2_cosine_recall': 0.9880715705765407,
 'paraphrase-multilingual-MiniLM-L12-v2_cosine_ap': 0.48367800657603427,
 'paraphrase-multilingual-MiniLM-L12-v2_dot_accuracy': 0.6071784646061814,
 'paraphrase-multilingual-MiniLM-L12-v2_dot_accuracy_threshold': 21.068126678466797,
 'paraphrase-multilingual-MiniLM-L12-v2_dot_f1': 0.6939058171745153,
 'paraphrase-multilingual-MiniLM-L12-v2_dot_f1_threshold': 13.113171577453613,
 'paraphrase-multilingual-MiniLM-L12-v2_dot_precision': 0.5324123273113709,
 'paraphrase-multilingual-MiniLM-L12-v2_dot_recall': 0.9960238568588

### Computing Metrics with `sklearn.metrics`

We use [scikit-learn metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) instead to compute our evaluation metrics.

In [32]:
# This will rapidly train the embedding model. MultipleNegativesRankingLoss did not work.
loss = losses.ContrastiveLoss(model=sbert_model)

sbert_args = SentenceTransformerTrainingArguments(
    output_dir=SBERT_OUTPUT_FOLDER,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_ratio=0.1,
    run_name=SBERT_MODEL,
    load_best_model_at_end=True,
    save_steps=SAVE_EVAL_STEPS,
    eval_steps=SAVE_EVAL_STEPS,
    save_strategy="steps",
    eval_strategy="steps",
    greater_is_better=False,
    metric_for_best_model="eval_loss",
    learning_rate=LEARNING_RATE,
    logging_dir="./logs",
)

trainer = SentenceTransformerTrainer(
    model=sbert_model,
    args=sbert_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=binary_acc_evaluator,
    compute_metrics=compute_sbert_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

trainer.evaluate()
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
wandb.finish()

### Try the Model from Our Best Epoch

We fine-tuned the model for `EPOCHS` nubmer of epochs, but the last epoch isn't always best. The `TrainingArgument` `load_best_model_at_end=True` loads the model at the end.

Another way to load the best model is to load our output folder and evaluate that `SentenceTransformer` on some examples to get a gestalt sense for its performance.

```python
sbert_model = SentenceTransformer(OUTPUT_FOLDER, device=device)
```

In [ ]:
sbert_compare(
    "101 Oak Lane, Atlanta, GA 30308",
    "102 Oak Lane, Atlanta, GA 30308",
)

In [ ]:
sbert_compare(
    "101 Oak Lane, Macon, GA 30308",
    "101 Oak Lane, Atlanta, GA 30408",
)

In [ ]:
sbert_compare(
    "101 Oak Lane, Atlanta, GA 30308",
    "101 Oak Ln., Atlanta, GA 30308",
)

In [ ]:
sbert_compare(
    "3413 Sean Way, Lawrenceville, GA 30044",
    "1202 Oak Rd., Lawrenceville, GA 30304",
)

In [ ]:
sbert_compare(
    "3413 Sean Way, Lawrenceville, GA 30044",
    "3413 Sean Way, Lawrenceville, GA 30044, USA",
)

### Evaluate ROC Curve to Determine Optimum Similarity Threshold

0.5 is an arbitrary line on which to divide positive (match, 1) and negative (mismatch, 0). Let's evaluate the ROC Curve of the F1 score to see what it should be set to. Recall that the `sbert_match` function has a `threshold: float = 0.5` argument.

#### Evaluate on our Augmented Test Dataset

First we'll evaluate the ROC curve on our augmented test dataset.

In [ ]:
y_true = test_df["Label"]
y_scores = test_df.apply(sbert_match, axis=1)

In [ ]:
# Compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

# Compute F1 score for each threshold
f1_scores = [f1_score(y_true, y_scores >= t) for t in thresholds]

# Find the threshold that maximizes the F1 score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f'Best Threshold: {best_threshold}')
print(f'Best F1 Score: {best_f1_score}')

roc_auc = roc_auc_score(y_true, y_scores)
print(f'AUC-ROC: {roc_auc}')

In [ ]:
# Create a DataFrame for Seaborn
pr_data = pd.DataFrame({
    'Precision': precision[:-1],
    'Recall': recall[:-1],
    'F1 Score': f1_scores
})

# Plot Precision-Recall curve using Seaborn
sns.lineplot(data=pr_data, x='Recall', y='Precision', marker='o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Augmented Test Set Precision-Recall Curve')
plt.show()

### Plot a ROC Curve for our Gold Labeled Data

We need to see the ROC Curve for our gold labeled data as well. We care more about performance on this data.

In [ ]:
y_true = gold_df["Label"]
y_scores = gold_df.apply(sbert_match, axis=1)

In [ ]:
# Compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

# Compute F1 score for each threshold
f1_scores = [f1_score(y_true, y_scores >= t) for t in thresholds]

# Find the threshold that maximizes the F1 score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f'Best Threshold: {best_threshold}')
print(f'Best F1 Score: {best_f1_score}')

roc_auc = roc_auc_score(y_true, y_scores)
print(f'AUC-ROC: {roc_auc}')

In [ ]:
# Create a DataFrame for Seaborn
pr_data = pd.DataFrame({
    'Precision': precision[:-1],
    'Recall': recall[:-1],
    'F1 Score': f1_scores
})

# Plot Precision-Recall curve using Seaborn
sns.lineplot(data=pr_data, x='Recall', y='Precision', marker='o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Gold Label Precision-Recall Curve')
plt.show()

### Debugging Errors on our Gold Labels

Let's evaluate the data using our `gold_label_report` function with the best F1 score. Then we can view the errors and figure out where our model is failing.

In [ ]:
raw_df, grouped_df = gold_label_report(
    gold_df,
    [
        strict_parse_match,
        parse_match_country,
        sbert_match_binary,
    ],
    threshold=best_threshold
)

#### Label Description Group Analysis

You can see the types of address pairs we are failing on. This can guide our data augmentation / programmatic labeling work at a high level.

In [ ]:
grouped_df.head(40)

In [ ]:
grouped_df["sbert_match_binary_acc"].sort_values().head(40)

#### What it Got Right ...

In [ ]:
# Truthiness analysis
correct_df = raw_df[raw_df["sbert_match_binary_correct"]].reset_index()
print(f"Number correct: {len(correct_df):,}")

correct_df.head(20)

In [ ]:
# Error analysis
wrong_df = raw_df[raw_df["sbert_match_binary_correct"] == False].reset_index()
print(f"Number wrong: {len(wrong_df):,}")

wrong_df.head(20)

# Fuzzy, Structured Address Matching with Libpostal and Vector Distance

There seems to be merit to both structured and embedding approaches to address matching. Let's see how an approximate approach to matching parsed address components might help.

## Combining an Address Parsing and Embedding Approach

Libpostal is a powerful parsing model, and it seems like parsing is a logical first step in matching addresses. Some logical combinations of fields should result in a match, while some should not. This logic can be programmed by a human, provided fuzzy matching is available at the field level. `101 Oak Lane` should match `101 Oak Ln` but not `102 Oak Lane`. `street_name` and `street_number` are separate fields in an address parsed by Libpostal. What is we got some machine learning help with field-level matching?

In [ ]:
sbert_compare("Oak Lane", "Oak Ln.")

In [ ]:
sbert_compare("Cheer Lane", "Cheer Road")

In [ ]:
sbert_compare("101", "110")

In [ ]:
sbert_compare("5th", "Fifth")

In [ ]:
sbert_compare("USA", "United States")

In [ ]:
sbert_compare("30044", "30308")

## Fine-Tuning a Lowercase `SentenceTransformer`

My first pass at this method did not work whatsoever - the performance of the matcher was abysmal. This was because Libpostal *lowercases* addresses when it parses them, and I did NOT do that to the training data on a first pass :) Once I did that and retrained below - things worked much better!

### Lowercase our Dataset

In [ ]:
lower_augment_results_df = augment_results_df.copy(deep=True)

lower_augment_results_df["Address1"] = lower_augment_results_df["Address1"].str.lower()
lower_augment_results_df["Address2"] = lower_augment_results_df["Address2"].str.lower()

In [ ]:
train_df, tmp_df = train_test_split(lower_augment_results_df, test_size=0.2, shuffle=True)
eval_df, test_df = train_test_split(tmp_df, test_size=0.5, random_state=42, shuffle=True)

train_dataset = Dataset.from_dict({
    "sentence1": train_df["Address1"].tolist(),
    "sentence2": train_df["Address2"].tolist(),
    "label": train_df["Label"].tolist(),
})

eval_dataset = Dataset.from_dict({
    "sentence1": eval_df["Address1"].tolist(),
    "sentence2": eval_df["Address2"].tolist(),
    "label": eval_df["Label"].tolist(),
})

test_dataset = Dataset.from_dict({
    "sentence1": test_df["Address1"].tolist(),
    "sentence2": test_df["Address2"].tolist(),
    "label": test_df["Label"].tolist(),
})

print(f"Training data:   {len(train_df):,}")
print(f"Validation data: {len(eval_df):,}")
print(f"Test data        {len(eval_df):,}")

In [ ]:
SBERT_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"
VARIANT = "lowercase"
MODEL_SAVE_NAME = (SBERT_MODEL + "-" + VARIANT).replace("/", "-")

EPOCHS = 6
BATCH_SIZE = 32
PATIENCE = 3
LEARNING_RATE = .00005
DATASET_MULTIPLE = CLONES_PER_RUN * RUNS_PER_EXAMPLE
SBERT_LOWER_OUTPUT_FOLDER = f"data/fine-tuned-sbert-{MODEL_SAVE_NAME}"
SAVE_EVAL_STEPS = 100

In [ ]:
# Initialize Weights & Biases
wandb.init(
    entity="rjurney",
    # set the wandb project where this run will be logged
    project="libpostal-reborn",
    # track hyperparameters and run metadata
    config={
        "variant": VARIANT,
        "dataset_multiple": DATASET_MULTIPLE,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "patience": PATIENCE,
        "learning_rate": LEARNING_RATE,
        "sbert_model": SBERT_MODEL,
        "sbert_output_folder": SBERT_LOWER_OUTPUT_FOLDER,
        "save_eval_steps": SAVE_EVAL_STEPS,
    },
)

In [ ]:
sbert_model_lower = SentenceTransformer(
    SBERT_MODEL,
    device=device,
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name=f"{SBERT_MODEL}-address-matcher-{VARIANT}",
    ),
)

In [ ]:
# Initialize the evaluator
binary_acc_evaluator = BinaryClassificationEvaluator(
    sentences1=eval_dataset["sentence1"],
    sentences2=eval_dataset["sentence2"],
    labels=eval_dataset["label"],
    name=SBERT_MODEL,
)
binary_acc_evaluator(sbert_model_lower)

In [ ]:
# This will rapidly train the embedding model. MultipleNegativesRankingLoss did not work.
loss = losses.ContrastiveLoss(model=sbert_model_lower)

sbert_args = SentenceTransformerTrainingArguments(
    output_dir=SBERT_LOWER_OUTPUT_FOLDER,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_ratio=0.1,
    run_name=SBERT_MODEL,
    load_best_model_at_end=True,
    save_steps=SAVE_EVAL_STEPS,
    eval_steps=SAVE_EVAL_STEPS,
    save_strategy="steps",
    eval_strategy="steps",
    greater_is_better=False,
    metric_for_best_model="eval_loss",
    learning_rate=LEARNING_RATE,
    logging_dir="./logs",
)

trainer = SentenceTransformerTrainer(
    model=sbert_model_lower,
    args=sbert_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=binary_acc_evaluator,
    compute_metrics=compute_sbert_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

trainer.evaluate()
trainer.train()

In [ ]:
trainer.evaluate()

### Rewrite our Matchers for Lowercase Duty

Need two versions of these to compare the original with their new lowercase cousins.

In [ ]:
def sbert_compare_lower(address1: str, address2: str) -> float:
    """sbert_compare - sentence encode each address into a fixed-length text embedding.
    Fixed-length means they can be compared with cosine similarity."""
    embedding1 = sbert_model_lower.encode(address1.lower())
    embedding2 = sbert_model_lower.encode(address2.lower())

    # Compute cosine similarity
    return 1 - distance.cosine(embedding1, embedding2)


def sbert_match_lower(row: pd.Series) -> pd.Series:
    """sbert_match - SentenceTransformer address matching, float iytoyt"""
    return sbert_compare_lower(row["Address1"], row["Address2"])


def sbert_compare_binary_lower(address1: str, address2: str, threshold: float = 0.5) -> Literal[0, 1]:
    """sbert_match - compare and return a binary match"""
    similarity = sbert_compare_lower(address1, address2)
    return 1 if similarity >= threshold else 0


def sbert_match_binary_lower(row: pd.Series, threshold: float = 0.5) -> pd.Series:
    """sbert_match_binary - SentenceTransformer address matching, binary output"""
    return sbert_compare_binary_lower(row["Address1"], row["Address2"], threshold=threshold)

### Evaluate ROC Curve to Determine Optimum Similarity Threshold

We need to evaluate the ROC Curve of the F1 score to see what it should be set to for our lowercase model too. Recall that the `sbert_match_lower` function has a `threshold: float = 0.5` argument.

#### Evaluate on our Augmented Test Dataset

First we'll evaluate the ROC curve on our augmented test dataset.

In [ ]:
y_true = test_df["Label"]
y_scores = test_df.apply(sbert_match_lower, axis=1)

In [ ]:
# Compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

# Compute F1 score for each threshold
f1_scores = [f1_score(y_true, y_scores >= t) for t in thresholds]

# Find the threshold that maximizes the F1 score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f'Best Threshold: {best_threshold}')
print(f'Best F1 Score: {best_f1_score}')

roc_auc = roc_auc_score(y_true, y_scores)
print(f'AUC-ROC: {roc_auc}')

In [ ]:
# Create a DataFrame for Seaborn
pr_data = pd.DataFrame({
    'Precision': precision[:-1],
    'Recall': recall[:-1],
    'F1 Score': f1_scores
})

# Plot Precision-Recall curve using Seaborn
sns.lineplot(data=pr_data, x='Recall', y='Precision', marker='o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Augmented Test Set Precision-Recall Curve')
plt.show()

### Plot a ROC Curve for our Gold Labeled Data

We need to see the ROC Curve for our gold labeled data as well. We care more about performance on this data.

In [ ]:
y_true = gold_df["Label"]
y_scores = gold_df.apply(sbert_match_lower, axis=1)

In [ ]:
# Compute precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_true, y_scores)

# Compute F1 score for each threshold
f1_scores = [f1_score(y_true, y_scores >= t) for t in thresholds]

# Find the threshold that maximizes the F1 score
best_threshold_index = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_index]
best_f1_score = f1_scores[best_threshold_index]

print(f'Best Threshold: {best_threshold}')
print(f'Best F1 Score: {best_f1_score}')

roc_auc = roc_auc_score(y_true, y_scores)
print(f'AUC-ROC: {roc_auc}')

In [ ]:
# Create a DataFrame for Seaborn
pr_data = pd.DataFrame({
    'Precision': precision[:-1],
    'Recall': recall[:-1],
    'F1 Score': f1_scores
})

# Plot Precision-Recall curve using Seaborn
sns.lineplot(data=pr_data, x='Recall', y='Precision', marker='o')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Gold Label Precision-Recall Curve')
plt.show()

### Rewriting our Structured Matcher to be Semantically Fuzzy

Let's rewrite our original parser to use `sbert_compare_binary(address1: str, address2: str, threshold=best_threshold)` for the road name, city (which we skipped before) and country.

In [ ]:
def parse_fuzzy_match_address(address1: str, address2: str, threshold=0.5) -> Literal[0, 1]:
    """parse_fuzzy_match_address uses parsed addresses with fuzzy matching for street, city and country."""
    address1 = to_dict(parse_address(address1))
    address2 = to_dict(parse_address(address2))

    def match_road(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_road - literal road matching, negative if either lacks a road"""
        if ("road" in address1) and ("road" in address2):

            # to_dict produces a list if two tuples have the same key
            if isinstance(address1["road"], list):
                address1["road"] = " ".join(address1["road"])
            if isinstance(address2["road"], list):
                address2["road"] = " ".join(address2["road"])
            
            if sbert_compare_binary_lower(
                address1["road"],
                address2["road"], 
                threshold=threshold
            ):
                logger.debug("road match")
                return 1
            else:
                logger.debug("road mismatch")
                return 0
        logger.debug("road mismatch")
        return 0

    def match_house_number(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_house_number - literal house number matching, negative if either lacks a house_number"""
        if ("house_number" in address1) and ("house_number" in address2):
            if address1["house_number"] == address2["house_number"]:
                logger.debug("house_number match")
                return 1
            else:
                logger.debug("house_number mismatch")
                return 0
        logger.debug("house_number mistmatch")
        return 0

    def match_unit(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_unit - note a missing unit in both is a match"""
        if "unit" in address1:
            if "unit" in address2:
                logger.debug("unit match")
                return 1 if (address1["unit"] == address2["unit"]) else 0
            else:
                logger.debug("unit mismatch")
                return 0
        if "unit" in address2:
            if "unit" in address1:
                logger.debug("unit match")
                return 1 if (address1["unit"] == address2["unit"]) else 0
            else:
                logger.debug("unit mismatch")
                return 0
        # Neither address has a unit, which is a default match
        return 1

    def match_postcode(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_postcode - literal matching, negative if either lacks a postal code"""
        if ("postcode" in address1) and ("postcode" in address2):
            if address1["postcode"] == address2["postcode"]:
                logger.debug("postcode match")
                return 1
            else:
                logger.debug("postcode mismatch")
                return 0
        logger.debug("postcode mismatch")
        return 0

    def match_country(address1: Dict, address2: Dict) -> Literal[0, 1]:
        """match_country - literal country matching - pass if both don't have one"""
        if ("country" in address1) and ("country" in address2):

            # to_dict produces a list if two tuples have the same key
            if isinstance(address1["country"], list):
                address1["country"] = " ".join(address1["country"])
            if isinstance(address2["country"], list):
                address2["country"] = " ".join(address2["country"])

            if sbert_compare_binary_lower(
                address1["country"],
                address2["country"],
                threshold=best_threshold,
            ):
                logger.debug("country match")
                return 1
            else:
                logger.debug("country mismatch")
                return 0

        # One or none countries should match
        logger.debug("country match")
        return 1

    # Combine the above to get a complete address matcher
    if (
        match_road(address1, address2)
        and match_house_number(address1, address2)
        and match_unit(address1, address2)
        and match_postcode(address1, address2)
        and match_country(address1, address2)
    ):
        logger.debug("overall match")
        return 1
    else:
        logger.debug("overall mismatch")
        return 0


def sbert_parse_match(row: pd.Series, threshold: float = 0.5) -> pd.Series:
    """fuzzy_parse_match Fuzzy, structured address matching. Threshold is passed through via gold_label_report."""
    return parse_fuzzy_match_address(row["Address1"], row["Address2"], threshold=threshold)

In [ ]:
raw_df, grouped_df = gold_label_report(
    gold_df,
    [
        strict_parse_match,
        parse_match_country,
        sbert_match_binary,
        sbert_parse_match,
    ],
    threshold=best_threshold
)

In [ ]:
grouped_df["sbert_parse_match_acc"].sort_values().head(40)

In [ ]:
# Truthiness analysis
correct_df = raw_df[raw_df["sbert_parse_match_correct"]].reset_index(drop=True)
print(f"Number correct: {len(correct_df):,}")

correct_df.head(20)

In [ ]:
# Error analysis
wrong_df = raw_df[raw_df["sbert_parse_match_correct"] == False].reset_index()
print(f"Number wrong: {len(wrong_df):,}")

wrong_df.head(20)

In [ ]:
sbert_compare_lower(
    "nw 5th ave",
    "northwest 5th avenue"
)

In [ ]:
parse_fuzzy_match_address(
    "2024 NW 5th Ave, Miami, FL 33127",
    "2024 Northwest 5th Avenue, Miami, Florida 33127",
    threshold=best_threshold
)

In [ ]:
parse_fuzzy_match_address(
    "Third Ave, New York, NY",
    "3rd Avenue, New York, New York"
)

In [ ]:
sbert_compare_lower(
    "101 market square, seattle, wa 98039",
    "101 davis place, seattle, wa 98039"
)

In [ ]:
parse_fuzzy_match_address(
    "221B Baker Street, London, NW1 6XE, UK",
    "221B Baker St, Marylebone, London NW1 6XE"
)

Not quite right... we didn't come close to beating sentence transformers alone. I think we really need a deep learning model doing the heavy lifting for us rather than a simple threshold on a binary classification.

## Fuzzy Parsed Conclusion

If you look at the items missed by this model, they are largely due to stricter matching requirements. These could be addressed logically, as with the help of fuzzy matching with `SentenceTransformers` they fall within the scope of task a human can accomplish. This means the structured, fuzzy model can be trusted more than the pure `SentenceTransformer` model. A little more work could bring it into a state that meets or beats the pure `SentenceTransformer` model while providing explainability that entity resolution often requires.

## Structured Prediction with a `Sentence-BERT` Classifier

Embeddings as a solution to this problem have a side-effect of optimizing an embedding for information retrieval... but they ignore the structure of parsed addresses. A deep network that is aware of it can perform better. Let's try out an implementation of the Sentence-BERT model, which was outlined by Nils Reimers and Iryna Gurevych in the original paper that created sentence tranformers, [Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks
](https://arxiv.org/abs/1908.10084).

In [33]:
SBERT_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# SBERT_MODEL = "data/fine-tuned-sbert-paraphrase-multilingual-MiniLM-L12-v2-original/checkpoint-2400/"
VARIANT = "untrained-embeddings"
MODEL_SAVE_NAME = ("Sentence-BERT" + "-" + VARIANT).replace("/", "-")

EPOCHS = 6
BATCH_SIZE = 32
PATIENCE = 3
LEARNING_RATE = 2e-5
MODEL_OUTPUT_FOLDER = f"data/{MODEL_SAVE_NAME}"
SAVE_EVAL_STEPS = 100

COLUMN_SPECIAL_CHAR = "[COL]"
VALUE_SPECIAL_CHAR = "[VAL]"

In [34]:
# Initialize Weights & Biases
wandb.init(
    entity="rjurney",
    # set the wandb project where this run will be logged
    project="libpostal-reborn",
    # track hyperparameters and run metadata
    config={
        "model": "Sentence-BERT",
        "variant": VARIANT,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "patience": PATIENCE,
        "learning_rate": LEARNING_RATE,
        "sbert_model": SBERT_OUTPUT_FOLDER,
        "model_output_folder": MODEL_OUTPUT_FOLDER,
        "save_eval_steps": SAVE_EVAL_STEPS,
        "model_save_name": MODEL_SAVE_NAME,
    },
)

eval/loss,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_cosine_accuracy,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_cosine_accuracy_threshold,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_cosine_ap,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_cosine_f1,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_cosine_f1_threshold,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_cosine_precision,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_cosine_recall,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_dot_accuracy,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_dot_accuracy_threshold,▁
eval/paraphrase-multilingual-MiniLM-L12-v2_dot_ap,▁


In [51]:
class SentenceBERT(nn.Module):
    def __init__(self, model_name=SBERT_MODEL, dim=384):
        super().__init__()
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        
        # Update the input dimension of the FFNN to account for the new features
        self.ffnn = nn.Sequential(
            nn.Linear(dim*5, dim),  # Changed from dim*4 to dim*5
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(dim, 1)
        )

    @staticmethod
    def mean_pool(token_embeds, attention_mask):
        in_mask = attention_mask.unsqueeze(-1).expand(token_embeds.size()).float()
        pool = torch.sum(token_embeds * in_mask, 1) / torch.clamp(in_mask.sum(1), min=1e-9)
        return pool

    def _check_similarity(self, a, b, mask_a, mask_b):
        u = self.model(a, attention_mask=mask_a)[0]
        v = self.model(b, attention_mask=mask_b)[0]
        u = self.mean_pool(u, mask_a)
        v = self.mean_pool(v, mask_b)
        
        uv = torch.abs(u - v)
        prod = u * v  # Element-wise product
        mult = torch.mul(u, v)  # Multiplication feature
        x = torch.cat([u, v, uv, prod, mult], dim=-1)  # Added prod and mult to the concatenation
        x = self.ffnn(x)
        return x

    def check_similarity(self, a, b):
        encoded_a = self.tokenizer(a, padding=True, truncation=True, return_tensors="pt")
        encoded_b = self.tokenizer(b, padding=True, truncation=True, return_tensors="pt")
        a = encoded_a["input_ids"]
        b = encoded_b["input_ids"]
        mask_a = encoded_a["attention_mask"]
        mask_b = encoded_b["attention_mask"]
        with torch.no_grad():
            return self._check_similarity(a, b, mask_a, mask_b)

    def forward(self, input_ids_a, input_ids_b, attention_mask_a=None, attention_mask_b=None, labels=None):
        logits = self._check_similarity(input_ids_a, input_ids_b, attention_mask_a, attention_mask_b)
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1), labels.float().view(-1))
        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

    def predict(self, a: str, b: str):
        with torch.no_grad():
            logits = self.check_similarity(a, b)
            probabilities = torch.sigmoid(logits)
            predicted_class = (probabilities > 0.5).long().item()
            return predicted_class, probabilities.item()

In [ ]:
classifier_model = SentenceBERT()

In [46]:
def structured_encode_address(address: str) -> str:
    """structured_parse_address - encode a parsed address"""
    parsed_address: List[Tuple[str, str]] = parse_address(address)
    sorted_address: List[Tuple[str, str]] = list(sorted(parsed_address, key=lambda x: x[1]))  # no secondary sort to maek it determinstic?
    encoded_address: str = str()
    for val, col in sorted_address:
        encoded_address += COLUMN_SPECIAL_CHAR + col + VALUE_SPECIAL_CHAR + val
    return encoded_address


def tokenize_function(examples):
    encoded_a = classifier_model.tokenizer(examples["sentence1"], padding="max_length", truncation=True)
    encoded_b = classifier_model.tokenizer(examples["sentence2"], padding="max_length", truncation=True)
    return {
        "input_ids_a": encoded_a["input_ids"],
        "attention_mask_a": encoded_a["attention_mask"],
        "input_ids_b": encoded_b["input_ids"],
        "attention_mask_b": encoded_b["attention_mask"],
        "labels": examples["label"]
    }


def format_dataset(dataset):
    dataset.set_format(type="torch", columns=["input_ids_a", "attention_mask_a", "input_ids_b", "attention_mask_b", "labels"])
    return dataset

In [47]:
train_df, tmp_df = train_test_split(augment_results_df, test_size=0.2, shuffle=True)
eval_df, test_df = train_test_split(tmp_df, test_size=0.5, random_state=42, shuffle=True)

# Encode the addresses using [COL] / [VAL] special characters
train_dataset = Dataset.from_dict({
    "sentence1": [structured_encode_address(x) for x in train_df["Address1"].tolist()],
    "sentence2": [structured_encode_address(x) for x in train_df["Address2"].tolist()],
    "label": train_df["Label"].tolist(),
})

# Encode the addresses using [COL] / [VAL] special characters
eval_dataset = Dataset.from_dict({
    "sentence1": [structured_encode_address(x) for x in eval_df["Address1"].tolist()],
    "sentence2": [structured_encode_address(x) for x in eval_df["Address2"].tolist()],
    "label": eval_df["Label"].tolist(),
})

# Encode the addresses using [COL] / [VAL] special characters
test_dataset = Dataset.from_dict({
    "sentence1": [structured_encode_address(x) for x in test_df["Address1"].tolist()],
    "sentence2": [structured_encode_address(x) for x in test_df["Address2"].tolist()],
    "label": test_df["Label"].tolist(),
})

print(f"Training data:   {len(train_df):,}")
print(f"Validation data: {len(eval_df):,}")
print(f"Test data        {len(eval_df):,}")

Training data:   8,024
Validation data: 1,003
Test data        1,003


In [48]:
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

tokenized_train_dataset = format_dataset(tokenized_train_dataset)
tokenized_eval_dataset = format_dataset(tokenized_eval_dataset)
tokenized_test_dataset = format_dataset(tokenized_test_dataset)

Map:   0%|          | 0/8024 [00:00<?, ? examples/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

Map:   0%|          | 0/1003 [00:00<?, ? examples/s]

In [49]:
def compute_classifier_metrics(eval_pred):
    logits, labels = eval_pred
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)
    predictions = (logits > 0.5).long().squeeze()
    
    print(f"Length of predictions: {len(predictions)}")
    print(f"Length of labels: {len(labels)}")
    
    if len(predictions) != len(labels):
        raise ValueError(f"Mismatch in lengths: predictions ({len(predictions)}) and labels ({len(labels)})")
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }

In [52]:
training_args = TrainingArguments(
    output_dir=f"data/{MODEL_SAVE_NAME}",
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_ratio=0.1,
    run_name=SBERT_MODEL,
    load_best_model_at_end=True,
    save_steps=SAVE_EVAL_STEPS,
    eval_steps=SAVE_EVAL_STEPS,
    save_strategy="steps",
    eval_strategy="steps",
    greater_is_better=False,
    metric_for_best_model="eval_loss",
    weight_decay=0.01,
    logging_dir="./logs",
)

class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_fct = torch.nn.BCEWithLogitsLoss()

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").float()
        input_ids_a = inputs.get("input_ids_a")
        attention_mask_a = inputs.get("attention_mask_a")
        input_ids_b = inputs.get("input_ids_b")
        attention_mask_b = inputs.get("attention_mask_b")
        outputs = model(input_ids_a, input_ids_b, attention_mask_a, attention_mask_b)
        loss = self.loss_fct(outputs['logits'].view(-1), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=classifier_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_classifier_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,No log,0.167067,0.937188,0.937562,0.946000,0.929273
200,No log,0.186201,0.925224,0.925669,0.934000,0.917485
300,No log,0.158629,0.948156,0.949318,0.941973,0.956778
400,No log,0.143667,0.952144,0.953216,0.945841,0.960707
500,0.140600,0.150601,0.950150,0.951172,0.945631,0.956778
600,0.140600,0.165611,0.948156,0.949119,0.945419,0.952849
700,0.140600,0.142971,0.949153,0.951009,0.930451,0.972495
800,0.140600,0.129564,0.964108,0.964637,0.964637,0.964637


Length of predictions: 1003
Length of labels: 1003
Length of predictions: 1003
Length of labels: 1003
Length of predictions: 1003
Length of labels: 1003
Length of predictions: 1003
Length of labels: 1003
Length of predictions: 1003
Length of labels: 1003
Length of predictions: 1003
Length of labels: 1003
Length of predictions: 1003
Length of labels: 1003
Length of predictions: 1003
Length of labels: 1003


KeyboardInterrupt: 

### ReWrite our Evaluation Code for the Model

In [ ]:
def classifier_match(row: pd.Series) -> pd.Series:
    """sbert_match - SentenceTransformer address matching, float iytoyt"""
    return model.predict(row["Address1"], row["Address2"])